# Projektarbeit Data Analytics

Sebastian Jana,
Sophie Jana

## Inhaltsverzeichnis
<a id ="inhaltsverzeichnis"></a>

[1. Aufgabe](#aufgabe1)

[2. Aufgabe](#aufgabe2)

[3. Aufgabe](#aufgabe3)

[4. Aufgabe](#aufgabe4)

[5. Aufgabe](#aufgabe5)

[6. Aufgabe](#aufgabe6)

[7. Quellenverzeichnis](#quellenverzeichnis)




### Aufgabe 1 (Datenvorbereitung)
<a id = "aufgabe1"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)


Vorab alle Imports die im Notebook benötigt werden:

In [ ]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import requests
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from wordcloud import WordCloud
import folium

a) Lesen Sie die CSV-Dateien, die die Stromerzeugungsdaten und Börsenstrompreise enthalten ein und führen Sie sie in einem DataFrame namens df_hourly zusammen.

Um die CSV-Dateien zu einem DataFrame zusammenführen, werden alle CSV-Dateien die sich im Ordner Strompreisdaten befinden eingelesen. Aufgrund der Umstellung zwischen Sommer- und Winterzeit unterschieden sich die Spaltennamen für das Datum "Datum (MESZ)" für die Sommerzeit und "Datum (MEZ)" für die Winterzeit. Um die Daten einheitlich zu analysieren, benennen wir die Datumsspalte in "DateTime" um. Abschließend werden die einzelnen DataFrames, die sich in df_list befinden, zeilenweise aneinandergereiht und bilden somit dann den Dataframe df_hourly.

In [ ]:
# Sources for reading csv files from one folder
# https://www.geeksforgeeks.org/how-to-read-all-csv-files-in-a-folder-in-pandas/
# https://statistikguru.de/python/python-auflisten-dateien-verzeichnis.html


path = './Daten/Strompreisdaten'
# List all files (.csv) in the given path
csv_files = glob.glob(os.path.join(path, "*csv"))

df_list = []
for i in range(len(csv_files)):
    try:
        df_temp = pd.read_csv(csv_files[i], sep = ",")
        for column in df_temp.columns:
            # Combine the date columns, by getting rid of the naming difference in csv source
            if 'Datum (MESZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MESZ)':'DateTime'})
            elif 'Datum (MEZ)' in column:
                df_temp = df_temp.rename(columns = {'Datum (MEZ)':'DateTime'})
        df_list.append(df_temp)
    except Exception as err: 
        print("Fehler beim Einlesen des Files: ", err)
    
df_hourly = pd.concat(df_list)


b) Passen Sie die dtypes der Spalten von df_hourly geeignet an. Überführen Sie insbesondere das Datum in ein DateTime-Format. Entfernen Sie alle Datensätze, die sich nicht auf den Betrachtungszeitraum 2020-2024 beziehen.

Zunächst haben wir die Datentypen der Spalten des DataFrames überprüft. Für die weitere Analyse eignet sich der ursprüngliche Datentyp float64 für die Spalten "Leistung nicht erneuerbar (MW)", "Leistung erneuerbar (MW)", und "Day Ahead Auktion Preis EUR/MWh", da die Daten numerische Werte darstellen.

Die Spalte "DateTime" wird in das datetime-Format umgewandelt, um mit Zeitstempeln arbeiten zu können. Dabei haben wir errors='coerce' verwendet, um ungültige Datumswerte in NaT (Not a Time) umzuwandeln, wobei nach der Überprüfung keine fehlerhaften DateTime Werte vorhanden sind. 
Anschließend beinhaltet der df_hourly nur den Zeitraum von 2020 bis 2024, sodass 48 Zeilen außerhalb des Zeitraums entfernt wurden. 

In [ ]:
print(df_hourly.shape)
# Converting the column "DateTime" from object to DateTime format
# Invalid values are converted to NaT (Not a Time)
# Source: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
df_hourly['DateTime'] = pd.to_datetime(df_hourly['DateTime'], errors="coerce")
print(df_hourly.dtypes)
print(df_hourly['DateTime'].isna().any())

# df_hourly contains only data for the obervation period
df_hourly = df_hourly[(df_hourly['DateTime'].dt.year >= 2020) & (df_hourly['DateTime'].dt.year <= 2024)]
print(df_hourly.shape) 
df_hourly = df_hourly.sort_values(by='DateTime')

c) Beurteilen Sie die Datenqualität des Datensatzes und führen Sie, soferen aus Ihrer Sicht notwendig, geeignete Datenbereinigungsschritte durch.

Nachdem wir festgestellt haben, dass keine fehlenden Werte in unseren Daten vorhanden sind, überprüfen wir, ob es aufgrund der Umstellung zwischen Sommer- und Winterzeit doppelte Einträge in der Spalte DateTime gibt. Wir identifizieren 28 Duplikate.

Um diese Duplikate zu bereinigen, gruppieren wir den DataFrame nach der DateTime-Spalte und berechnen den Durchschnitt für die Spalten. Nur für Duplikate wird daraus dann eine gemeinsame Zeile.

Der anschließende reset_index() stellt sicher, dass DateTime wieder eine reguläre Spalte im DataFrame ist und nicht als Index verwendet wird.



Die abschließende Beurteilung der Datenqualität zeigt, dass die Daten nach den durchgeführten Bereinigungs- und Transformationsschritten für die weitere Analyse vorbereitet wurden. Es wurden keine fehlenden Werte festgestellt, was die Vollständigkeit der Daten gewährleistet. Doppelte Einträge, die durch die Umstellung zwischen Sommer- und Winterzeit entstanden sind, wurden bereinigt.
Zudem wurde die DateTime-Spalte in ein datetime-Format überführt, wodurch zeitbezogene Berechnungen und Analysen effizient durchgeführt werden können. Der DataFrame wurde auf den Zeitraum von 2020 bis 2024 eingeschränkt, sodass ausschließlich relevante Daten berücksichtigt werden.

In [ ]:
# Check for missing values
print(df_hourly.isnull().sum())
print(df_hourly.notnull().sum())

# Doppelungen DateTime (28 Einträge; 14 Doppelungen)
df_check = df_hourly.groupby('DateTime').size()
duplicates = df_check[df_check > 1]
duplicates_df = df_hourly[df_hourly['DateTime'].isin(duplicates.index)]
duplicates_df.shape

# Doppelungen der DateTime bereinigen 
df_hourly = df_hourly.groupby('DateTime').mean().reset_index()
df_hourly


d) Erzeugen Sie aus df_hourly einen weiteren DataFrame namens df_daily, der in jeder Zeile die erzeugte elektrische Energie mit erneuerbaren und nicht erneuerbaren Energieträgern sowie an diesem Tag durchschnittlich gemessenen Börsenstrompreis enthält.

Um die Aufgabenstellung zu erfüllen, erstellen wir aus df_hourly einen neuen DataFrame df_daily, der die tägliche Summe der erzeugten erneuerbaren Energie und die tägliche Summe der nicht erneuerbaren Energie sowie den durchschnittlichen Börsenstrompreis enthält.

Um die Orginaldaten nicht zu verändern, erstellen wir zuerst eine Kopie des df_hourly.
Anschließend wird das Datum ohne Uhrzeit aus der DateTime-Spalte extrahiert und in einer neuen Spalte Datum gespeichert, um eine Gruppierung auf Tagesebene zu ermöglichen.

Abschließend führen wir die Aggregationen über einen left-Join auf der Spalte Datum zusammen, wobei der Index zurückgesetzt wird.

Die Spalte Datum überführen wir anschließend wieder in das datetime-Format zurück, um weiterhin zeitbezogenen Berechnungen und Filterungen durchführen zu können.

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
df_daily = df_hourly.copy()

# Create column "Date" from column "DateTime"
df_daily['Datum'] = df_daily['DateTime'].dt.date
# Grouping by the date without time
df_power = df_daily.groupby('Datum')[['Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)' ]].sum()

df_mean = df_daily.groupby('Datum')['Day Ahead Auktion Preis (EUR/MWh)'].mean().reset_index()

df_daily = pd.merge(df_power, df_mean, on='Datum', how='left')

# Change Datum again to datetime format
df_daily['Datum'] = pd.to_datetime(df_daily['Datum'])

df_daily


### Aufgabe 2 (Explorative Analyse der Stromerzeugungs- und Preisdaten)
<a id = "aufgabe2"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)


a) An welchen 10 Tagen im Betrachtungszeitraum wurde am meisten Strom aus erneuerbaren Energieträgern erzeugt?

Um die 10 Tage mit der höchsten Stromerzeugung aus erneuerbaren Energieträgern im Betrachtungszeitraum zu ermitteln, sortieren wir den DataFrame df_daily nach der Spalte "Leistung erneuerbar (MW)" in absteigender Reihenfolge. Dadurch erscheinen die Tage mit den höchsten Werten an der Spitze.

Anschließend verwenden wir die Methode .head(10), um die ersten zehn Zeilen des sortierten DataFrames auszuwählen. Diese enthalten die Tage mit der höchsten Stromerzeugung aus erneuerbarer Energie.

Sichtbar wird dabei, dass die 10 Tage mit der höchsten Stromerzeugung aus erneuerbaren Energien in Deutschland in die Wintermonate fallen. 
Eine mögliche Erklärung liefert die Zusammensetzung der erneuerbaren Energieträger in Deutschland. 
Die Windkraft ist hierbei die stärkste Quelle der erneuerbaren Energieträgern.
https://www.umweltbundesamt.de/themen/klima-energie/erneuerbare-energien/erneuerbare-energien-in-zahlen#waerme

Im Winter sind die Winde aufgrund größerer Temperatur- und Druckunterschiede besonders stark, was zu einer hohen Auslastung der Windkraftanlagen führt.
https://klimavest.de/de/wissen/blog/winterzeit-ist-windzeit/#:~:text=Windenergie%20erklärt,auch%20der%20Ertrag%20der%20Photovoltaikanlagen

Unsere weiterführende Überlegung ist dann, dass es sich bei den Tagen mit expliziten Höchstwerten um Sturmereignisse in Deutschland handelt. 
Tatsächlich lässt sich feststellen, dass für alle diese Daten starke Stürme über Deutschland gezogen sind. 


Die beiden höchsten Tage sind der 05.06.2024 und der 06.02.2024, wo es ein Sturmtief in der Nordhälfe Deutschlands gab.
https://www.tagesschau.de/wetter/wetterthema/2024-02-05-windspitzen-102.html

24.1.24: Sturm Jitka 
https://www.dwd.de/DE/wetter/thema_des_tages/2024/1/25.html#:~:text=Mit%20Sturmtief%20JITKA%20schwang%20sich,Kern%20%C3%BCber%20der%20Norwegischen%20See

Allgemein war der Februar 2024 ein Rekordmonat der Windenergiegewinnung.
https://www.ingenieur.de/technik/fachbereiche/energie/windkraft-in-deutschland-2024-ein-rekordjahr-im-ueberblick/
https://www.agora-energiewende.de/daten-tools/agorameter-update-der-deutsche-strommix-im-februar-2024

21.12.23 und 22.12.23 und 29.12.23: Orkantief Zoltan
https://www.tagesschau.de/inland/gesellschaft/sturmtief-zoltan-auswirkungen-100.html
https://t3n.de/news/sturmtief-zoltan-rekordwerte-windenergiegewinnung-1598472/

21.12.23 Höchstwert Stormerzeugung Windenergie 2023
https://www.ingenieur.de/technik/fachbereiche/energie/windenergie-energiequelle-des-jahres/

07.04.2022: Sturmtief Nasim
https://www.merkur.de/deutschland/dwd-prognose-april-unwetter-deutschland-sturmtief-nasim-wind-sturm-zr-91463248.html
17.02.2022: 
https://www.dwd.de/DE/wetter/thema_des_tages/2022/2/17.html

11.02.2020 Orkantief Sabine

https://www.stern.de/panorama/wetter/sturmtief--sabine---60-prozent-sturmstrom---sabine--treibt-produktion-von-windenergie-kraeftig-an-9126944.html



In [ ]:
ten_highest_days = df_daily.sort_values(by = 'Leistung erneuerbar (MW)', ascending=False).head(10)
ten_highest_days

b) An welchem Tag im Betrachtungszeitraum wurde der höchste Börsenstrompreis verzeichnet und wie hoch war er? An welchem Tag wurde der geringste Preis verzeichnet und wie hoch war er?

Zur Ausgabe des Tages mit dem höchsten Börsenstrompreis und des Tages mit dem niedrigsten Börsenstrompreis verwenden wir die max() und min() Methode über die Spalte "Day Ahead Auktion Preis (EUR/MWh)". Anschließend haben wir den dazugehörigen Tag ermittelt. Zur intuitiveren Ausgabe extrahieren wir aus der DateTime nur das Datum ohne die Uhrzeit. Für die Ausgabe der Börsenstrompreise runden wir auf 2 Nachkommastellen.

In [ ]:
highest_price = df_daily['Day Ahead Auktion Preis (EUR/MWh)'].max()
date_highest_price = df_daily[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] == highest_price]

print("Tag mit dem höchsten Börsenstrompreis: " , date_highest_price['Datum'].dt.date.iloc[0])
print("Höchster Börsenstrompreis: ", round(highest_price, 2))

lowest_price = df_daily['Day Ahead Auktion Preis (EUR/MWh)'].min()
date_lowest_price = (df_daily.loc[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] == lowest_price])
print("Tag mit dem niedrigsten Börsenstrompreis: ", date_lowest_price['Datum'].dt.date.iloc[0])
print("Niedrigster Börsenstrompreis: ", round(lowest_price, 2))


In [ ]:
df_day_highest_and_lowest_price = df_daily[(df_daily['Datum'] == '2023-07-02') | (df_daily['Datum'] == '2022-08-26')]
df_day_highest_and_lowest_price

Zum besseren Verständis haben wir uns grundlegend über das Zustandekommen des Börsenstrompreis informiert. Bei dem Day-Ahead-Markt wird Strom gehandelt, der am nächsten Tag geliefert wird. Händler geben hierbei an der Börse ihre Gebote auf der Grundlage von Prognosen für die Stromnachfrage und die verfügbaren Energieerzeugung ab. 
https://www.rabot.energy/magazin/boersenstrompreis/

Die günstige Erzeugung der Energie hat dabei Vorrang. Preise reflektieren hiermit die Erzeugungskosten, also die Grenzkosten einer Stromeinheit.
Wenn der gesamte Strombedarf gedeckt ist steht der Preis fest. Ausschlaggebend ist die Anlage mit den höchsten Grenzkosten, die noch Strom einspeist.
Nicht erneuerbare wie Kohne oder Gas haben höhere Grenzkosten.
https://www.smard.de/page/home/wiki-article/446/384

Mit erneuerbaren Energien gibt es günstigere Preise aber auch höhere Schwankungen auf dem Strommarkt, da die diese Energieträger von externen Einflüssen abhängen. 
https://www.mdr.de/wissen/umwelt-klima/so-veraendern-eneuerbare-energien-den-Strommarkt100.html


Niedrige Börsenstrompreise entstehen, wenn das Stromangebot durch z.B. erneuerbare Energiequellen die Nachfrage übersteigt. 
https://www.rabot.energy/magazin/boersenstrompreis/


Der niedrigste durchschnittliche Börsenstrompreis pro Tag war am 02.07.2023 mit -53,87 EUR/MWh. Grund für den niedrigen Preis war ein hohes Angebot an Solar- und Windenergie, sowie ein geringer Stromverbrauch am Wochenende. 

https://www.next-kraftwerke.de/energie-blog/energiemonat-juli-2023
ttps://de.statista.com/statistik/daten/studie/1536358/umfrage/niedrigste-strompreise-am-epex-spotmarkt-in-deutschland/


Der höchste durschnittliche Börsenstrompreis pro Tag war am 26.08.2022 mit 699,44 EUR/MWh. Hintergrund war die Energiekrise 2022 durch den Angriffskrieg Russlands gegen die Ukraine und die damit verbundenen Engpässe, Unsicherheiten und Preissteigerungen. Im Lauf des Jahres 2022 wurde als Reaktion auf die Sanktionen immer weniger Gas von Russland nach Deutschland exportiert, wobei am 31.08.2022 endgültiger Stopp war, was zu einem Anstieg der Preise für Gas führte. 
https://www.eon.com/de/innovation/zukunft-der-energie/leben-und-kommunen/wie-sich-der-ukraine-krieg-auf-die-energiepreise-auswirkt.html
https://de.statista.com/statistik/daten/studie/1316029/umfrage/russischer-gasexport-nach-deutschland-auf-tagesbasis/#:~:text=Zwischen%20dem%2011.07


Am 26.08.22 erreichte der Gaspreis den Höchstand, was unter anderem den hohen Börsenstrompreis erklärt.
https://www.ffe.de/veroeffentlichungen/veraenderungen-der-merit-order-und-deren-auswirkungen-auf-den-strompreis/



Zur Verdeutlichung der Energiekrise 2022 visualisieren wir ebenfalls den mittleren Börsenstrompreis pro Tag für den Zeitraum 2020-2024. Deutlich sichtbar wird hierbei der starke Anstieg Ende 2021 bis Ende 2022, sowie einige extreme Preisspitzen. 

In [ ]:
# "Line chart to show the distribution of prices per day for the observation period

plt.figure(figsize=(15, 6))
plt.plot(df_daily['Datum'], df_daily['Day Ahead Auktion Preis (EUR/MWh)'], color = 'blue')
plt.xlabel('Betrachtungszeitrum (2020-2024)')
plt.ylabel('Mittlerer Börsenstrompreise in €/MWH pro Tag')
plt.title('Mittlerer Börsenstrompreis pro Tag für 2020-2024')
plt.show()

c) Wie viele Tagen gab es im Betrachtungszeitraum 2020-2024, an denen ein negativer Börsenstrompreis aufgetreten ist?

Zur Analyse der Tage, welche einen negativen Börsenstrompreis aufweisen, geben wir alle Zeilen aus die einen Börsenstrompreis pro Tag kleienr 0 haben. Im Betrachtungszeitraum ist das an 13 Tagen der Fall gewesen. Auf den 07.02.2022 wurde schon in Aufgabe 2 b) eingegangen.
Wie in Aufgabe 2b) beschrieben kommt es zu negativen Börsenstrompreisen, wenn durch z.B. erneuerbare Energiequellen die Nachfrage übersteigt. 
Sichtbar wird hier, dass an diesen Tagen die Leistung der erneuerbarer Energieträger sehr stark war und die Leistung aus nicht erneuerbaren Energieträgern relativ niedrig war.


In [ ]:
negative_price_df = df_daily[df_daily['Day Ahead Auktion Preis (EUR/MWh)'] < 0]
negative_price_day_count = negative_price_df['Day Ahead Auktion Preis (EUR/MWh)'].count()
print("Anzahl der Tage mit einem negativen Börsenstrompreis: ", negative_price_day_count)
negative_price_df


d) Wie viel Strom wurde pro Jahr mit erneuerbaren und mit nicht erneuerbaren Energieträgern erzeugt?

Um den den erzeugten Strom pro Jahr zu analysieren extrahieren wir das Jahr als eigene Spalte, da wir das Jahr als eigene Spalte für weitere Analysezwecke für sinnvoll halten. 
Anschließend gruppieren nach Jahr und aggregieren die Leistung der erneuerbaren und nicht erneuerbaren Energieträgern als Summe pro Jahr. Zur anschaulicheren Ausgabe rechnen wir die Leistung in Gigawatt um und runden auf 2 Nachkommastellen.

Zur besseren Einordnung berechnen wir noch die Summe der gesamten erzeugten Stroms in GW und stellen dann den Anteil der erneuerbaren und den Anteil der nicht erneuerbaren an der Gesamtleistung in Prozent dar. 

Sichtbar wird ein Anstieg der Leistung aus erneuerbaren Energieträgern und ein Rückgang der Leistung aus nicht erneuerbaren Energieträgern.
Diese Ergebnisse spiegeln den aktuellen Strommix in Deutschland wieder, wobei rund die Hälfte des Stroms schon aus erneuerbaren Energiequellen stammt. 
https://www.ndr.de/nachrichten/info/Strommix-Deutschland-Wie-ist-der-Anteil-erneuerbarer-Energien,strommix102.html

Es fällt deutlich auf dass es von 2020 nach 2021 einen leichten Anstieg der nicht erneuerbaren Energieträger gibt und einen leichten Abfall der erneuerbaren Energiequellen. Seit 2021 ist aber der Trend zu erkennen, dass die nicht erneuerbaren zurückgehen und die erneuerbaren ansteigen.


In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.year.html#pandas.Series.dt.year
df_daily['Jahr'] = df_daily['Datum'].dt.year
# convert year to int for better indexing later on
df_daily['Jahr'] = df_daily['Jahr'].astype(int)

df_year_power = df_daily.groupby('Jahr')[['Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)']].sum()

#Convert from megawatts (MW) to gigawatts (GW)

df_year_power['Leistung nicht erneuerbar (GW)'] = (df_year_power['Leistung nicht erneuerbar (MW)']/1000).round(2)
df_year_power['Leistung erneuerbar (GW)'] = (df_year_power['Leistung erneuerbar (MW)']/1000).round(2)
df_year_power


# Prozentuale Darstellung des Anteils der erneuerbaren Energie am gesamten erzeugten Strom

df_year_power['Summe erzeugter Strom in GW'] = df_year_power['Leistung erneuerbar (GW)'] + df_year_power['Leistung nicht erneuerbar (GW)']

df_year_power['Anteil erneuerbar an Gesamtleistung in %'] = ((df_year_power['Leistung erneuerbar (GW)'] / df_year_power['Summe erzeugter Strom in GW']) * 100).round(2)
df_year_power['Anteil nicht erneuerbar an Gesamtleistung in %'] = 100 - df_year_power['Anteil erneuerbar an Gesamtleistung in %']

df_year_power



## Aufgabe 3 (Weiterführende Analyse der Stromerzeugung- und Preisdaten)
<a id = "aufgabe3"></a>


[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)


a) Visualisieren Sie in geeigneten Diagrammen die Verteilung der Börsenstrompreise (insgesamt und pro Jahr).


Zur Visualisierung der Verteilung der Börsenstrompreise über den gesamten Betrachtungszeitraum haben wir uns für ein Histogramm entschieden, da dieses gut geeignet ist darzustellen, wie häufig gewisse Börsenstrompreise in verschiedenen Preisspannen auftreten. 

Zur anschaulicheren Unterteilung wählen wir die Intervalle manuell mit einem Abstand von 50 Euro. Die Analyse des Histogramms zeigt hierbei, dass ein Großteil der Börsenstrompreise zwischen 0 und 100 Euro pro MWh liegen.
Preise über 200 Euro pro MWh kommen deutlich seltener vor. Vereinzelte Extremwerte liegen zwischen -100 und -50 und ab 400 bis zum maximalen Börsenstrompreis. 

Die Verteilunng ist rechtsschief. Das bedeutet, dass sich die meisten Börsenstrompreise auf den niedrigen Bereich von 0 bis 100 Euro pro MWh konzentrieren. 


In [ ]:
x= df_daily['Day Ahead Auktion Preis (EUR/MWh)']
plt.figure(figsize=(15, 6))

intervalle = [-100, -50, 0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700]
plt.hist(x, bins=intervalle , edgecolor='k')

plt.title('Verteilung der mittleren Börsenstrompreise pro Tag von 2020-2024')
plt.ylabel('Anzahl')
plt.xlabel('Mittlerer Börsenstrompreis in EUR/MWh')

plt.xticks(intervalle)

plt.show()


Zur Visualisierung der Verteilung der Börsenstrompreise pro Jahr wird ein Boxplot verwendet, da dieser einen übersichtlichen Vergleich zwischen den Jahren und den statistischen Kenngrößen ermöglicht.

Die Analyse zeigt, dass der Median der Strompreise im Jahr 2022 stark über dem Niveau der anderen Jahre liegt, was die Auswirkungen der Energiekrise 2022 verdeutlicht. Der Median der übrigen Jahre unterscheidet sich nur geringfügig. Auffällig ist zudem, dass das Maximum im Jahr 2022 signifikant über den Höchstwerten der anderen Jahre liegt. Der Interquartilsabstand (IQR) ist im Jahr 2022 deutlich größer, was auf eine stärkere Streuung der mittleren Börsenstrompreise in diesem Zeitraum hinweist.

Nach 2022 ist eine Rückkehr der Strompreise auf das Niveau der Jahre 2020 und 2021 erkennbar, was auf eine Marktstabilisierung hindeutet.

Ein weiterer Aspekt sind die Ausreißer: Während 2021 und 2024 vermehrt Ausreißer nach oben aufweisen, treten 2020 und 2023 vermehrt Ausreißer nach unten auf.

In [ ]:
df_daily.boxplot(column = 'Day Ahead Auktion Preis (EUR/MWh)', by = 'Jahr', grid = False, figsize = (15,6))

plt.ylabel("Börsenstrompreis EUR/MWh")
plt.title('Verteilung der täglichen mittleren Börsenstrompreise pro Jahr')
plt.suptitle('')
plt.show()


b) Berechnen Sie bezogen auf die einzelnen Jahre des Betrachtungszeitraums verschiedene statistische Kenngrößen für den Börsenstrompreis.

Zur Berechnung statistischer Kenngrößen pro Jahr für den Börsenstrompreis gruppieren wir den DataFrame nach dem Jahr und verwenden dann die Methode .describe(), wobei für jedes Jahr die Anzahl, der Mittelwert, die Standardabweichung, das Minimum, das Maximum, das 25% Quartil, das 50% Quartil (der Median) und das 70% Quartil berechnet werden. 

Wir erkennnen das Maximum und das Minimum des DataFrames aus der Aufgabe 2b).
Die Anzahl zeigt, dass es sich bei den Jahren 2020 und 2024 um Schaltjahre handelt, was wir bei der weiteren Analyse berücksichtigen werden. 
Der im Boxplot zuvor erkennbare Median wird hier als genauer Wert sichtbar. Im Vergleich zu 2021 und 2022 ist dieser im Jarh 2022 mehr als doppelt so hoch. Ein ähnliches Verhalten zeigt sich beim Mittelwert für 2022. Die Standardabweichung für das Jahr 2022 ist im Vergleich zu den anderen Jahren ebenso signifikant höher, was die Auswirkungen der Energiekrise wiederspiegelt. Wir erkennen, dass die Streuung in den anderen Jahren deutlich geringer ist, was auf einen stabileren Börsenstrompreis hindeutet. 
Auffällig ist die vorallem die geringe Standardabweichung im Jahr 2020, was auf einen stabileren Börsenstrompreis in diesem Jahr hindeutet. 


In [ ]:
df_year_statistics = df_daily.groupby('Jahr')['Day Ahead Auktion Preis (EUR/MWh)']
df_year_statistics.describe()


c) Visualisieren Sie in einem Säulendiagramm die mittleren Börsenstrompreise pro Monat des Betrachtungszeitraums.

Zur Visualisierung des mittleren Börsenstrompreises pro Monat erzeugen wir aus der DateTime Spalte eine Spalte mit dem Monat. Anschließend gruppieren wir nach Jahr und Monat, um den Börsenstrompreis über den gesamten Betrachtungszeitraum im Laufe der Zeit zu visualisieren. 
Das Säulendiagramm spiegelt die Erkenntnisse aus den vorherigen Teilaufgaben anschaulich wieder.

In [ ]:
df_daily['Monat'] = df_daily['Datum'].dt.month
# converting string to int 
df_daily['Monat'] = df_daily['Monat'].astype(int)

df_year_monthly = df_daily.groupby(['Monat'])['Day Ahead Auktion Preis (EUR/MWh)'].mean()

df_year_monthly.plot(kind = 'bar',figsize=(10, 6))

plt.ylim(0,200)
plt.xticks(rotation=0)

plt.title('Mittlerer Börsenstrompreis pro Monat von 2020-2024')
plt.ylabel('Mittlerer Börsenstrompreis in EUR/MWh')

plt.show()

In [ ]:
df_daily.boxplot(column = 'Day Ahead Auktion Preis (EUR/MWh)', by = 'Monat', grid = False, figsize = (15,6))

plt.ylabel("Börsenstrompreis EUR/MWh")
plt.xlabel('Monate')
plt.title('Verteilung der täglichen mittleren Börsenstrompreise pro Monat')
# Entfernt den Titel, der durch "by" gesetzt wird
plt.suptitle('')
plt.show()


d) Visualisieren Sie die stündlichen Börsenstrompreise in einem interaktiven Liniendiagramm in Plotly. Versehen Sie dieses mit einem Range-Selektor und einem Range-Slider. Analysieren Sie auf der Basis der Ergebnisse der Teilaufgaben a)-d) die wesentlichen Entwicklungen und Trends der Börsenstrompreise im Betrachtungszeitraum 2020-2024.

In [ ]:
# https://plotly.com/python/range-slider/

fig = go.Figure()
fig.add_trace(go.Scatter 
              (x = df_hourly['DateTime'],
              y = df_hourly['Day Ahead Auktion Preis (EUR/MWh)'],
              mode = 'lines'))


fig.update_layout(
    
    title_text="Stündlicher Börsenstrompreis von 2020-2024",
    ## Disables automatic scaling
    autosize=False, 
    width=1400,      
    height=500,
    yaxis_title= "Börsenstrompreis EUR/MWh",
    xaxis_title = "Datum und Uhrzeit",
    
    
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(step='all', label= 'Gesamt'),
                dict(count=1,
                     label="letzter Monat",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="letzten 6 Monate",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="letztes Jahr",
                     step="year",
                     stepmode="backward"),              
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        
        type="date"
    ), 
    
    # https://plotly.com/python/dropdowns/  

    updatemenus = [
        dict(
            buttons = [
                
                dict(label= 'Alle Jahre', method ='relayout', args=[{"xaxis.range": ["2020-01-01 00:00:00", "2024-12-31 23:00:00"]}]),
                dict(label="2020",
                    method="relayout",
                    args=[{"xaxis.range": ["2020-01-01 00:00:00", "2020-12-31 23:00:00"]}]),
                dict(label="2021",
                    method="relayout",
                    args=[{"xaxis.range": ["2021-01-01 00:00:00", "2021-12-31 23:00:00"]}]),
                dict(label="2022",
                    method="relayout",
                    args=[{"xaxis.range": ["2022-01-01 00:00:00", "2022-12-31 23:00:00"]}]),
                dict(label="2023",
                    method="relayout",
                    args=[{"xaxis.range": ["2023-01-01 00:00:00", "2023-12-31 23:00:00"]}]),
                dict(label="2024",
                    method="relayout",
                    args=[{"xaxis.range": ["2024-01-01 00:00:00", "2024-12-31 23:00:00"]}]),
            ],
            
            x=1.04,
            xanchor='left',
            y=1,  
            yanchor='top'
            
            
            
        )
    ] 
     
)
fig.show()

e) Berechnen und visualisieren Sie die im Mittel mit erneuerbaren Energieträgern erzeugten Energie im Tagesverlauf, indem Sie auf die vollen Stunden eines Tages aggregieren.

Grundlage für die visualisierung der im Mittel mit erneuerbaren Energieträgern erzeugten Energie im Tagesverlauf ist der Dataframe df_hourly. Zur weiteren Analyse extrahieren wir anschließend aus der Spalte "DateTime" die Uhrzeit, sowie den Monat. Um die mittlere stündliche Leistung zu berechnen, gruppieren wir die Daten nach der Uhrzeit und ermitteln den Mittelwert der Leistung der erneuerbaren Energieträger in MW für jede Stunde des Tages. Anschließend visualisieren wir die Ergebnisse in einem Liniendiagramm, um den Verlauf über einen Tag hinweg am besten darzustellen. Die x-Achse repräsentiert dabei die Stunden eines Tages und die y-Achse die mittlere erzeugte Leistung aus erneuerbaren Energieträgern.

Wir erkennen, dass sich nachts die erzeugte erneuerbare Energie auf einem relativ konstanten Niveau befindet. Tagsüber ab 8:00 Uhr steigt die Leistung stark an und erreicht zwischen 12:00 und 13:00 das Maximum. Anschließend nimmt die Leistung der erneuerbaren Energieträgern wieder ab. Dies spiegelt den Himmelsstand der Sonne wieder und zeigt den Einfluss, den Photovoltaikanlagen haben. 

Auf Basis der bisherigen Erkenntnisse möchten wir genauer untersuchen, wie sich der Tagesverlauf der erneuerbaren Energieerzeugung zwischen Sommer- und Wintermonaten unterscheidet. Da die Intensität der Sonnenstrahlung im Winter geringer ist erwarten wir deutliche Unterschiede im Tagesverlauf der Leistung der erneuerbaren Energieträgern.
Wir unterteilen dafür unseren Dataframe in Sommermonate (Juni, Juli, August) und Wintermonate (Januar, Februar, Dezember).

Es zeigt sich, dass im Tagesverlauf der Wintermonate die Leistung der erneuerbaren Energieträger kaum schwankt und nur einen leichten Anstieg in der Mitte des Tages aufweist. Im Gegensatz dazu zeigt die Leistung der erneuerbaren Energieträger im Sommer einen deutlichen stärkeren Anstieg und ebenso starken Abstieg im Laufe des Tages.

Diese Erkenntnis verdeutlicht den deutschen Stormmix aus erneuerbaren Energiequellen. So liefern Windräder im Winter mehr Strom als im Sommer, wobei die Leistung der Windkraft im Tagesverlauf konstanter ist. Die erneuerbare Energie im Sommer schwankt deutlich abhängig von dem Stand der Sonne im Tagesverlauf. 
https://www.ndr.de/nachrichten/info/Strommix-Deutschland-Wie-ist-der-Anteil-erneuerbarer-Energien,strommix102.html


In [ ]:
df_hourly_month_and_time = df_hourly.copy()
df_hourly_month_and_time['Monat'] = df_hourly_month_and_time['DateTime'].dt.month
df_hourly_month_and_time['Uhrzeit'] = df_hourly_month_and_time['DateTime'].dt.hour

df_hourly_mean= df_hourly_month_and_time.groupby('Uhrzeit')['Leistung erneuerbar (MW)'].mean().reset_index()


plt.figure(figsize=(15, 6))
plt.plot(df_hourly_mean['Uhrzeit'], df_hourly_mean['Leistung erneuerbar (MW)'], color = 'green')
plt.xlabel('Uhrzeit')
plt.ylabel('Mittlere erzeugte Leistung in der Stunde (MW)')
plt.title('Durchschnittlicher Tagesverlauf erneuerbarer Energie')


time = ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", 
    "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", 
    "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"]

# https://queirozf.com/entries/matplotlib-examples-number-formatting-for-axes-labels

plt.gca().set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])
plt.gca().set_xticklabels(time)


# einheitliche y-Achse
plt.ylim(0,50000)

plt.show()


In [ ]:

# Sommermonate Juni, Juli, August
summer_months = [6,7,8]
# Wintermonate Januar, Februar, Dezember
winter_months = [1,2,12]

df_summer = df_hourly_month_and_time.where(df_hourly_month_and_time['Monat'].isin(summer_months)).reset_index()
df_summer = df_summer.dropna()
df_summer = df_summer.groupby('Uhrzeit')['Leistung erneuerbar (MW)'].mean().reset_index()

df_winter = df_hourly_month_and_time.where(df_hourly_month_and_time['Monat'].isin(winter_months)).reset_index()
df_winter = df_winter.dropna()
df_winter = df_winter.groupby('Uhrzeit')['Leistung erneuerbar (MW)'].mean().reset_index()



plt.figure(figsize=(15, 6))
plt.plot(df_summer['Uhrzeit'], df_summer['Leistung erneuerbar (MW)'], color = 'orange', label ='Sommer')
plt.plot(df_winter['Uhrzeit'], df_winter['Leistung erneuerbar (MW)'], color = 'blue', label = 'Winter')

plt.xlabel('Uhrzeit')
plt.ylabel('Mittlere erzeugte Leistung in der Stunde (MW)')
plt.title('Durchschnittlicher Tagesverlauf erneuerbarer Energie im Sommer (Juni, Juli, August) und Winter (Dezember, Januar, Februar)')
plt.legend()
plt.gca().set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])
plt.gca().set_xticklabels(time)
# einheitliche y-Achse
plt.ylim(0,50000)

plt.show()

f) Visualisieren Sie in einem geeigneten Diagramm die pro Tag mit erneuerbaren Energieträgern erzeugte elektrische Energie und analysieren Sie diese. Gehen Sie dabei sowohl auf einzelne auffällige Tage als auch auf übergeordnete Entwicklungen und Trends ein.

Zur Darstellung der pro Tag mit erneuerbaren Energieträgern erzeugten elektrischen Energie haben wir uns für eine interaktive Heatmap mit Plotly entschieden, da man so den übergeordneten Trend der Jahre erkennen kann, aber ebenso die genaue Leistung pro Tag angezeigt werden kann.

Unser Ziel ist es auf der x-Achse die Tage eines Jahres abzubilden und auf der y-Achse die 5 Jahre des Betrachtungszeitraums übereinander.

Wir extrahieren für die Darstellung auf der x-Achse die Tage für ein gesamtes Jahr, indem wir ein Datum ohne das Jahr erzeugen. Um anschließend für den Plot wieder das datetime-Format nutzen zu können fügen wir dem Datum ein beliebiges Standardjahr hinzu und überführen diese neue Datum wieder in das datetime-Format. 

Um die Tage eines Jahres für jedes Jahr mit dem entsprechendnen Leistung für die erneuerbaren darzustellen erzeugen wir eine Pivottabelle mit den Jahren als Index und als Spalten die Tage mit dem Standardjahr. 

Die in Aufgabe 2b) gezeigten Schaltjahre haben 366 Tage. Deswegen füllen wir die fehlenden Tage in den anderen Jahren mit einem Mittelwert über den gesamten Betrachtungszeitraum auf. 

In [ ]:
df_heatmap = df_daily.copy()
df_heatmap['Leistung erneuerbar (GW)'] = df_heatmap['Leistung erneuerbar (MW)']/1000
df_heatmap['Leistung erneuerbar (GW)'] = df_heatmap['Leistung erneuerbar (GW)'].round(2)

#https://www.geeksforgeeks.org/pandas-series-dt-strftime/
df_heatmap['Datum ohne Jahr'] = df_heatmap['Datum'].dt.strftime("%m-%d")
df_heatmap['Datum einheitliches Jahr'] = '1980-' + df_heatmap['Datum ohne Jahr']

# Umwandeln in datetime (mit Platzhalterjahr)
df_heatmap['Datum einheitliches Jahr'] = pd.to_datetime(df_heatmap['Datum einheitliches Jahr'])

df_heatmap_pivot = df_heatmap.pivot(index='Jahr', columns='Datum einheitliches Jahr', values='Leistung erneuerbar (GW)')

mean_erneuerbar = df_heatmap['Leistung erneuerbar (GW)'].mean()
df_heatmap_pivot = df_heatmap_pivot.fillna(mean_erneuerbar)
df_heatmap_pivot

trace = go.Heatmap(
        z = df_heatmap_pivot,
        x = df_heatmap_pivot.columns,
        y = df_heatmap_pivot.index,
        colorscale='magma',
        colorbar= dict(title = 'Leistung in GW'),
        hovertemplate =
        'Datum: %{x|%d.%m} <br>' 
        'Energie: %{z} GW',
        
        name=''
    )
data = [trace]
months = ['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']

# Spaltenindex der Starttage der Monate
positionen = [0, 31, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335] 
# x Werte (DateTime-Format) für die Tick-Positionen verwenden
tickvals = []
for i in positionen: 
    tickvals.append(df_heatmap_pivot.columns[i])

layout = go.Layout(
    title=go.layout.Title(
        text='Erzeugte erneuerbare Energie pro Tag für die Jahre 2020-2024 ',
    ),
    # https://plotly.com/python/reference/layout/xaxis/#layout-xaxis
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Monate',
        ),
        tickmode='array', 
        tickvals= tickvals,
        ticktext=months,   
    ),
    
    yaxis= go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text= 'Jahre',
        )
    )
)

fig = go.Figure(data=data, layout=layout)
fig.show()


Auffällige hohe Tage:
06.02.2024 und 05.02.2024: Auffällige hohe Leistung in GW. Wie in Aufgabe 2 a) zu sehen sind diese beiden Tage die Tage mit der höchsten täglichen Leistung der erneuerbaren Energieträgern.
16.02 - 25.02. 2022

11.03.2021 und 12.03.2021

26.08.2020

20.12.2023- 29.12.2023: Aufgabe 2 a) Höchstwerte Stromerzeugung aus erneuerbar 2023 

20.10.21-22.10.21


Auffällig niedrige Tage:
20.01.2023- 28.01.2023

07.01.2021 - 09.01 2021

16.07.2020: 

9.12 - 12.12.22


Übergeordneter Trend: 
Die Darstellung als Heatmap zeigt deutlich, dass in den Wintermonaten eines Jahres die Stromerzeugung aus erneuerbaren Energieträgern größeren Schwankungen unterliegt. Im Vergleich zu den Sommermonaten erkennt man, dass es viele Tage mit einer hohen Leistung gibt, aber auch viele Tage mit einer sehr niedrigen Erzeugung. In den Sommermonaten hingegen sind keine starken Leistungsspitzen zu sehen, dafür aber auch keine besonders niedrigen Tage. Das verdeutlicht, dass die Erzeugung erneuerbarer Energie im Sommer konstanter ist.

Zur anschaulicheren Darstellung des übergeordneten Trends erzeugen wir einen zusätzlichen Boxplot mit den mittleren Börsenstrompreisen pro Jahr. 
Hierbei lassen sich die vorherigen Beobachtungen bestätigen. In den Wintermonaten ist der Median höher, ebenso aber auch der Interquartilsabstand und somit die Streuung der erzeugten Energie aus erneuerbaren Energieträgern. In den Wintermonaten sind die maximalen Werte auch höher. In den Sommermonaten wiederum ist der Median etwas niedriger, aber die Streuung geringer. 


In [ ]:
df_power_grouped_by_month = df_daily.copy()
df_power_grouped_by_month['Leistung erneuerbar (GW)'] = df_power_grouped_by_month['Leistung erneuerbar (MW)']/1000
df_power_grouped_by_month.boxplot(column = 'Leistung erneuerbar (GW)', by = 'Monat', grid = False, figsize = (15,6))

plt.ylabel("Leistung erneuerbar (GW)")
plt.xlabel('Monate')
plt.title('Verteilung der täglichen erneuerbaren Leistung pro Monat')
# Entfernt den Titel, der durch "by" gesetzt wird
plt.suptitle('')
plt.show()

g) Visualisieren Sie auf geeingete Weise die Zusammensetztung des erzeugten Stroms (erneuerbar vs. nicht erneuerbar) im Zeitverlauf und analysieren Sie diese.

Zur Visualisierung der Zusammensetzung des erzeugten Stroms nutzen wir ein gestapeltes Balkendiagramm, um den Anteil der erneuerbaren und nicht erneuerbaren Leistung an der berechneten Gesamtleistung darzustellen. 
Als Zeitverlauf haben wir uns für die Jahre des Betrachtungszeitraums entschieden, um den übergeordneten Trend des Strommixes darzustellen. Der Plot spiegelt die gewonnenen Erkenntisse aus Aufgabe 2 d) wieder. Es zeigt sich ein leichter Anstieg der erneuerbaren und ein leichter Rückgang der nicht erneuerbaren ab 2022.


In [ ]:
plt.figure(figsize=(15,6))

y2 = df_year_power['Anteil nicht erneuerbar an Gesamtleistung in %']
y1 = df_year_power['Anteil erneuerbar an Gesamtleistung in %']
x = df_year_power.index
p1 = plt.bar(x, y2, bottom = y1, label = 'Nicht Erneuerbar', color = 'grey')
p2 = plt.bar(x, y1, label ='Erneuerbar', color = 'green')

plt.ylabel('Anteil an der erzeugten Gesamtleistung')
plt.xlabel('Jahre')
plt.title('Zusammensetzung des erzeugten Stroms (erneuerbar vs. nicht erneuerbar) von 2020-2024')

# oberen linken Eckpunkt der Legende an die Koordinaten setzen
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()


### Aufgabe 4 (Untersuchung von Einflussfaktoren auf den Strompreis)
<a id = "aufgabe4"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a) Implementieren Sie eine Funktion namens
get_weather_data(lat, lon, start_date, end_date), die die (tagesbezogenen) Wetterdaten für die durch (lon,lat) gegebene Geo-Position im Zeitraum zwischen start_date und end_date von der open-meteo-API bezieht und das Ergebnis als DataFrame zurückgibt. Verwenden Sie für den Zugriff auf die API das Paket requests. Jede Zeile des DataFrames soll die Wetterdaten zu einem Tag enthalten. Wenden Sie diese Funktion anschließend an, um historische Wetterdaten der Jahre 2020 - 2024 für die Stadt Amberg zu beziehen. Reichern Sie den DataFrame df_daily um diese Wetterdaten an. Speichern Sie den resultierenden Datensatz in einer CSV-Datei namens daily.csv und laden Sie diese mit Ihrer Abgabe auf Moodle hoch.

Wir haben uns für stündliche Daten entschieden, weil wir damit auch überprüfen können, ob z.B. die Durchschnittstemperatur an einem Tag besser oder schlechter mit dem Strompreis korreliert als die Maximaltemperatur.

Allgemein haben wir die Wetterparameter per API geholt, bei denen wir vermutet haben, dass sie eine Korrleation mit Wind/Solar-Energieerzeugung haben.

In [ ]:
def extract_weather_data_to_df(api_json):
    # Get the hourly data from the JSON-Response
    hourly_data = api_json.get('hourly')
    time = hourly_data.get('time')

    # Get the data from the dict and create a DataFrame
    df = pd.DataFrame({
            'Time': time,
            'Temperatur °C': hourly_data.get('temperature_2m'),
            'Bewölkung %': hourly_data.get('cloud_cover'),
            'Sonnenscheinenergie W/m2' : hourly_data.get('direct_radiation'),
            'Windgeschwindigkeit km/h': hourly_data.get('wind_speed_100m'),
            'Niederschlag mm': hourly_data.get('rain'),
            'Atmosphärendruck hPa': hourly_data.get('pressure_msl')
        })
    
    df['Time'] = pd.to_datetime(df['Time'], errors="coerce")
    df['Datum'] = df['Time'].dt.date

    # Group DataFrame by date -- mean/sum values
    df_daily_mean = df.groupby('Datum').agg({
        'Temperatur °C': 'mean',      
        'Bewölkung %': 'mean',             
        'Sonnenscheinenergie W/m2': 'mean',  
        'Windgeschwindigkeit km/h': 'mean',
        'Niederschlag mm': 'sum',
        'Atmosphärendruck hPa': 'mean'
    }).reset_index()

    # Group DataFrame by date -- max values
    df_daily_max = df.groupby('Datum').agg({
        'Temperatur °C': 'max',                   
        'Windgeschwindigkeit km/h': 'max'
    }).reset_index()

    # Rename to be able to merge
    df_daily_max = df_daily_max.rename(columns = {'Temperatur °C' : 'Max Temperatur °C', 'Windgeschwindigkeit km/h' : 'Max Windgeschwindigkeit km/h'})

    # Combine the two DataFrames 
    df_combined = pd.merge(df_daily_mean, df_daily_max, on = 'Datum', how = 'left', validate='one_to_one')
    df_combined['Datum'] = pd.to_datetime(df_combined['Datum'], errors="coerce")

    return df_combined

def get_weather_data(lat, lon, start_date, end_date):
    # Fetch hourly data from the API, which we later group for the day
    params = {
        'latitude': lat,
        'longitude': lon,
        # Same Timezone as df_daily/df_hourly (Default is GMT (Greenwich Mean Time), so CET-1)
        'timezone': 'CET',
        'start_date': start_date,
        'end_date': end_date,
        'hourly': 'temperature_2m,cloud_cover,direct_radiation,wind_speed_100m,rain,pressure_msl'
    }
    weather_api_url = 'https://archive-api.open-meteo.com/v1/archive'

    api_answer = requests.get(weather_api_url, params)

    # If status code is 200, the API responded with a valid JSON
    if api_answer.status_code == 200:
        data = api_answer.json()
        return extract_weather_data_to_df(data)
    else:
       print('Error fetching Api data!')
       # empty DataFrame
       return pd.DataFrame()

# Coordinates of Digital Campus OTH Amberg
LAT_AMBERG = 49.44539774004614
LON_AMBERG = 11.84824284729236

df_weather_daily = get_weather_data(LAT_AMBERG, LON_AMBERG, '2020-01-01', '2024-12-31')
df_weather_daily

In [ ]:
# Only filter to relevant columns
df_daily = df_daily[['Datum', 'Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)', 'Day Ahead Auktion Preis (EUR/MWh)']]

# Check if API failed / DataFrame is empty
if(not df_weather_daily.empty):
    # Merge Weather Data and df_daily
    df_daily = pd.merge(df_daily, df_weather_daily, on = 'Datum', how = 'left', validate='one_to_one')
    df_daily.to_csv('daily.csv', index = False, encoding='utf-8')
df_daily

b) Reichern Sie den DataFrame df daily weiterhin um Börsenschlusskurse für Kohle und Erdgas an, die in den CSV-Dateien gaspreise.csv bzw. kohlepreise.csv gegeben sind. Visualisieren und untersuchen Sie auf geeignete Weise die Zusammenhänge (paarweise) zwischen Börsenstrompreis, Gaspreis, Kohlepreis und erneuerbarer Energieerzeugung.

In [ ]:
# Read in Gas
df_gas = pd.read_csv('./Daten/gaspreise.csv', sep = ",")
df_gas = df_gas.rename(columns = {'Schlusskurs' : 'Schlusskurs Gas'})
df_gas['Datum'] = pd.to_datetime(df_gas['Datum'], errors="coerce")

# Read in Kohle
df_kohle = pd.read_csv('./Daten/kohlepreise.csv', sep = ",")
df_kohle = df_kohle.rename(columns = {'Schlusskurs' : 'Schlusskurs Kohle'})
df_kohle['Datum'] = pd.to_datetime(df_kohle['Datum'], errors="coerce")

# Merge into df_daily
df_daily = pd.merge(df_daily, df_gas, on = 'Datum', how = 'left', validate='one_to_one')
df_daily = pd.merge(df_daily, df_kohle, on = 'Datum', how = 'left', validate='one_to_one')
df_daily

Zusammenhänge (paarweise) zwischen Börsenstrompreis, Gaspreis, Kohlepreis und erneuerbarer Energieerzeugung

In [ ]:
plt.scatter(df_daily['Day Ahead Auktion Preis (EUR/MWh)'], df_daily['Leistung erneuerbar (MW)'])
plt.title("Zusammenhang Börsenstrompreis und erneuerbarer Energieerzeugung")
plt.xlabel("Börsenstrompreis")
plt.ylabel("Leistung erneuerbar (MW)")
plt.show()

Es gibt vor allem dann Ausreißer nach oben im Börsenstrompreis, wenn nicht so viel erneuerbare Energie erzeugt wird.

In [ ]:
plt.scatter(df_daily['Day Ahead Auktion Preis (EUR/MWh)'], df_daily['Schlusskurs Kohle'])
plt.title("Zusammenhang Börsenstrompreis und Kohlepreis")
plt.xlabel("Börsenstrompreis")
plt.ylabel("Schlusskurs Kohle")
plt.show()

Wie schon in der 3) thematisiert, sind die nicht erneuerbaren häufig die Haupteinflüsse auf den Börsenstrompreis. Das zeigt sich auch hier wieder in dem Plot, da ein starker Zusammenhang zwischen hohem Kohlepreis und hohem Börsenstrompreis zu sehen ist.

In [ ]:
plt.scatter(df_daily['Day Ahead Auktion Preis (EUR/MWh)'], df_daily['Schlusskurs Gas'])
plt.title("Zusammenhang Börsenstrompreis und Gaspreis")
plt.xlabel("Börsenstrompreis")
plt.ylabel("Schlusskurs Gas")
plt.show()

Der selbe Zusammenhang wie bei Kohle schon zu sehen war, tritt auch bei Gas wieder auf. Auch der Kohlepreis lenkt als nicht erneuerbare Energieform den Börsenstrompreis

In [ ]:
plt.scatter(df_daily['Schlusskurs Gas'], df_daily['Leistung erneuerbar (MW)'])
plt.title("Zusammenhang Gaspreis und erneuerbarer Energieerzeugung")
plt.xlabel("Gaspreis")
plt.ylabel("Leistung erneuerbar (MW)")
plt.show()

Es tritt ein ähnlicher Zusammenhang auf wie schon beim Börsenstrompreis zu erneuerbarer Energieerzeugung, jedoch nur nicht so stark. Das war auch vorhersehbar, da wie schon vorher gezeigt, ein hoher Gaspreis tendenziell zu einem hohen Börsenstrompreis führt und somit diese beide stark von einander abhängen.

In [ ]:
plt.scatter(df_daily['Schlusskurs Kohle'], df_daily['Leistung erneuerbar (MW)'])
plt.title("Zusammenhang Kohlepreis und erneuerbarer Energieerzeugung")
plt.xlabel("Kohlepreis")
plt.ylabel("Leistung erneuerbar (MW)")
plt.show()

Der Vergleich zwischen Kohlepreis und erneuerbarer Energieerzeugung unterstreicht noch mal, den Zusammenhang der schon zwischen fossilen Brennstoffpreisen und erneuerbarer Erzeugung beobachtet wurde.

In [ ]:
plt.scatter(df_daily['Schlusskurs Gas'], df_daily['Schlusskurs Kohle'])
plt.title("Zusammenhang Kohlepreis und Gaspreis")
plt.xlabel("Gaspreis")
plt.ylabel("Kohlepreis")
plt.show()

Der Kohle- und Gaspreis zeigen einen starken Zusammenhang. Das erklärt sich aus der Nachfrage nach den beiden Ressourcen. Wenn, wie 2022, Gas knapp (und damit teuer) wird, gibt es dementsprechend natürlich eine höhere Nachfrage nach anderen fossilen Brennstoffen, um den Strombedarf weiterhin zu decken, was dann in letzter Konsequenz den Kohlepreis ebenfalls ansteigen lässt.

In [ ]:
df_gas_coal = df_daily.copy()

# Fill gaps (weekends/holidays)
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html
df_gas_coal['Schlusskurs Gas'] = df_gas_coal['Schlusskurs Gas'].ffill()
df_gas_coal['Schlusskurs Kohle'] = df_gas_coal['Schlusskurs Kohle'].ffill()

fig, ax1 = plt.subplots(figsize=(20, 7))

# https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html#sphx-glr-gallery-subplots-axes-and-figures-two-scales-py
# First y-axis (Gas)
ax1.plot(df_gas_coal['Datum'], df_gas_coal['Schlusskurs Gas'], 'r-', label='Gaspreis')
ax1.set_xlabel('Datum')
ax1.set_ylabel('Schlusskurs Gas', color='red')

# Second y-axis (Kohle)
ax2 = ax1.twinx()  # Create a twin y-axis
ax2.plot(df_gas_coal['Datum'], df_gas_coal['Schlusskurs Kohle'], 'b-', label='Kohlepreis')
ax2.set_xlabel('Datum')
ax2.set_ylabel('Schlusskurs Kohle', color='blue')

# Add a title
plt.title('Zeitlicher Verlauf des Gas- und Kohlepreis')

# Display the plot
plt.show()

In diesem Diagramm sieht man nochmal deutlich, wie direkt der Kohle- und Gaspreis, wegen dem oben beschriebenen Effekt, zusammenhängen.

c) Führen Sie eine Korrelationsanalyse für alle Variablen des DataFrames df_daily durch.
Erzeugen Sie dazu eine interaktive Correlation HeatMap in Plotly.

Wir erzeugen eine Korrelationsmatrix über die .corr() Methode und zeigen diese dann in einer Heatmap. Dabei stehen sowohl sehr helle als auch sehr dunkle Felder für starke Korrleation der Variablen/Spalten.

In [ ]:
df_corr = df_daily.copy()
# Filter out date based columns for the correlation matrix
df_corr_matrix = df_daily.copy()
df_corr_matrix = df_corr_matrix.drop(columns = ['Datum'])

correlation_matrix = df_corr_matrix.corr()

trace = go.Heatmap(
        z = correlation_matrix,
        x = correlation_matrix.columns,
        y = correlation_matrix.index,
        colorscale='magma',
        colorbar= dict(title = 'Korrelation'),
        hovertemplate =
        'Korrelation: %{z}',
        name=''
    )
data = [trace]

layout = go.Layout(
    title=go.layout.Title(
        text='Korrelation aller Variablen in df_daily',
    ),
    width = 1000,
    height = 800,
)

fig = go.Figure(data=data, layout=layout)
fig.show()


Positive Korrelationen:

Der Börsenstrompreis zeigt eine starke positive Korrelation mit dem Schlusskurs Gas und eine etwas geringere Korrelation mit dem Schlusskurs Kohle. Dies unterstreicht die Bedeutung fossiler Brennstoffe als wesentliche Einflussfaktoren für die Strompreisbildung, was bereits in vorherigen Analysen beschrieben wurde.

Eine moderate positive Korrelation ist zudem bei der Leistung nicht erneuerbarer Energien (MW) erkennbar. Dies deutet darauf hin, dass höhere Strompreise häufig mit einer verstärkten Nutzung fossiler Energiequellen einhergehen, während erneuerbare Energien in diesen Phasen weniger stark genutzt werden.
<br></br>
<br></br>
Negative Korrelationen:

Der Börsenstrompreis zeigt eine deutliche negative Korrelation zur Leistung erneuerbarer Energien (MW). Dies verdeutlicht, dass ein höheres Angebot an erneuerbaren Energien, insbesondere aus Wind- und Solarenergie, den Strompreis tendenziell senkt – ein Zusammenhang, der bereits in vorherigen Scatterplots erkennbar war.

Eine schwache negative Korrelation ist außerdem bei den wetterabhängigen Größen wie Sonnenscheindauer und Windgeschwindigkeit erkennbar, da diese Variablen direkt die Einspeisung erneuerbarer Energien beeinflussen. Dabei sticht jedoch die Windgeschwindigkeit (km/h) als der entscheidendste Faktor hervor, was die Bedeutung der Windkraft für den Strompreis erneut bestätigt.

Interessanterweise zeigen selbst indirekte Indikatoren für starke Winde, wie Niederschlag (mm) und Atmosphärendruck (hPa), eine stärkere Korrelation mit dem Börsenstrompreis als direkte Indikatoren für eine gute Solarstromerzeugung, wie Sonnenscheinenergie (W/m²) oder Temperatur (°C). Das zeigt noch einmal, dass Deutschlands erneuerbare Energieerzeugung viel stärker von Windenenergie abhängt als von Solar, was uns auch so vor dem Projekt gar nicht bewusst war.

### Aufgabe 5 (Modellbildung)
<a id = "aufgabe5"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a) Erstellen Sie unter Verwendung der Bibliothek Scikit-learn ein lineares Regressionsmodell zur Modellierung des mittleren Börsenstrompreises pro Tag in Abhängigkeit verschiedener Eingangsgrößen. Selektieren Sie basierend auf der vorherigen Aufgabe zunächst geeignete Merkmale als Eingangsgrößen. Teilen Sie die Daten anschließend in eine Trainings und eine Testdatenmenge und erstellen Sie ein lineares Regressionsmodell auf dem Trainingsdatensatz. Wie lautet der ermittelte funktionale Zusammenhang zwischen den Input und der Outputgrößen des Modells?

Als wichtigste Fakorten fließen der Preis von Kohle und Gas in das Modell ein, darüber hinaus auch die Menge an erzeugtem Strom (erneuerbar/nicht erneuerbar). Aus der Aufgabe 4 kann abgelesen werden, dass das Wetter nicht unbedingt stark mit dem Strompreis korreliert, dennoch ist vor allem die Windgeschwindigkeit ein potenzieller Indikator für billigen Strom.

Auch beziehen wir Monat/Tag/Wochentag in das Modell mit ein, um gewisse Trends des Preises auf Monatseben, sowie auch auf Tagesebene (Börsenstart Montag, Börsenschluss Freitag für Kohle und Gas) widerspiegeln.

In [ ]:
# Creating Date based indicators
df_corr['Monat'] = df_daily['Datum'].dt.month
df_corr['Monat'] = df_corr['Monat'].astype(int)

df_corr['Tag'] = df_daily['Datum'].dt.day
df_corr['Tag'] = df_corr['Tag'].astype(int)

df_corr['Wochentag'] = df_daily['Datum'].dt.day_of_week
df_corr['Wochentag'] = df_corr['Wochentag'].astype(int)

# Gas and Kohle are missing values for the days the exchange market is closed on
# Filling with Moving Average Mean of the 5 Workdays before that
df_corr['Moving_Avg'] = df_corr['Schlusskurs Kohle'].rolling(window=5).mean()
df_corr['Schlusskurs Kohle'] = df_corr['Schlusskurs Kohle'].fillna(df_corr['Moving_Avg'])
# If Moving Average doesn't result in a value, fill with the overall mean
df_corr['Schlusskurs Kohle'] = df_corr['Schlusskurs Kohle'].fillna(df_corr['Schlusskurs Kohle'].mean())


df_corr['Moving_Avg'] = df_corr['Schlusskurs Gas'].rolling(window=5).mean()
df_corr['Schlusskurs Gas'] = df_corr['Schlusskurs Gas'].fillna(df_corr['Moving_Avg'])
df_corr['Schlusskurs Gas'] = df_corr['Schlusskurs Kohle'].fillna(df_corr['Schlusskurs Gas'].mean())

# Including Month/Day as Number in the parameters, it's hard to correlate, but possibly still connected to the energy price 
X = df_corr[['Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)', 'Schlusskurs Gas', 'Schlusskurs Kohle', 'Windgeschwindigkeit km/h', 'Temperatur °C', 'Niederschlag mm', 'Atmosphärendruck hPa', 'Monat', 'Tag', 'Wochentag']]  
y = df_corr['Day Ahead Auktion Preis (EUR/MWh)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model = LinearRegression()

model.fit(X_train, y_train)

# Predict on the training data split
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("Koeffizienten:", model.coef_)  
print("R² Wert:", r2)


Die Koeffizienten beschreiben, in welchem Verhältnis die verschiedenen Parameter den Strompreis berechnen. Ein positiver Koeffizient zeigt an, dass ein Anstieg dieses Parameters den Strompreis erhöht, während ein negativer Koeffizient auf eine Verringerung des Strompreis hinweist.

Zusätzlich geben wir noch den R²-Wert für das Modell an, der beschreibt, wie gut das Modell die Variabilität des Strompreises erklärt. Ein R²-Wert nahe 1 bedeutet, dass das Modell fast alle Schwankungen im Strompreis durch die Input-Parameter erklären kann, während ein niedriger R²-Wert darauf hinweist, dass wesentliche Einflussfaktoren im Modell fehlen oder die Daten sehr stark streuen.

Unser Modell schwankt dabei zwischen 0.68 und 0.8, und bietet damit eine doch gute Grundlage für Analyse und die Prognose des Strompreises. 

Testcode für die Simulation von beliebig vielen Läufen:

In [ ]:
LINEAR_REGRESSION_RUNS = 5_000
'''
mean_r2 = 0
mean_mape = 0
min_mape = 1_000_000_000
max_mape = 0
for i in range(LINEAR_REGRESSION_RUNS):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    model = LinearRegression()

    model.fit(X_train, y_train)

    # Predict on the training data split
    y_pred = model.predict(X_test)

    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    min_mape = min(min_mape, mape)
    max_mape = max(max_mape, mape)
    mean_mape = (mape+mean_mape)/2
    mean_r2 = (r2+mean_r2)/2

print("Minimaler MAPE: " + str(min_mape))
print("Maximaler MAPE: " + str(max_mape))
print("Durchschnittlicher MAPE: " + str(mean_mape))
print("Durchschnittlicher R²: " + str(mean_r2))
'''

Output bei 5.000 Läufen:

Minimaler MAPE: 0.3615034526472132

Maximaler MAPE: 1.3154197668086212

Durchschnittlicher MAPE: 0.7193967384730049

Durchschnittlicher R²: 0.7604182190783234

b) Beurteilen Sie die Güte des Modells, indem Sie den mittleren relativen Fehler (mean absolute percentage error, verfügbar in sklearn.metrics) berechnen und auswerten.

Der MAPE schwankt je nach Ausführung zwischen 0,4 und 1,3. Das ist dementsprechend eine Abweichung von 60 % bis 120 % vom realen Strompreis an der Börse.

Bei einem hochvolatilen Markt wie dem Strommarkt kann eine so hohe Abweichung durchaus Sinn ergeben, da der Preis von vielen Faktoren beeinflusst wird, die man kaum vollständig ins Modell aufnehmen kann. Beispiele sind:

1) Geopolitische Ereignisse (z. B. Krisen, Sanktionen, Lieferengpässe), die kurzfristig zu starken Preisänderungen bei Brennstoffen wie Gas oder Kohle führen können.
<br></br>
2) Unvorhersehbare Kraftwerksausfälle oder Wartungsarbeiten, die das Angebot abrupt reduzieren.
<br></br>
3) Wetterextreme (z. B. kalte Winter, Hitzewellen, Sturmfluten), die gleichzeitig die Nachfrage (Heiz-/Kühlbedarf) und das Angebot (Wind-/Solarausfall) beeinflussen.
<br></br>
4) Kurzfristige Nachfragepeaks durch hohen Stromverbrauch

https://temagazin.de/regenerative-energien/strompreise-wie-entstehen-sie-und-warum-sind-sie-so-volatil/

In [ ]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Mean Absolute Percentage Error (MAPE):", mape)

c) Gehen Sie auf mögliche Limitierungen Ihres Modells ein.

Das Modell hat einige Limitierungen:
<br></br>
1) Durch die Energiekrise 2022, die dem Modell auch zugrunde liegt, kennt das Modell eine Varianz für höhere Strompreise, die in der Form heute wahrscheinlich erstmal nicht wieder auftreten.
<br></br>
2) Das Modell kennt nur die Wetterdaten von Amberg, die natürlich nicht direkt repräsentiv für die gesamtdeutsche Stromerzeugung aus Erneuerbaren steht.
<br></br>
3) Das Modell kennt keine detailreiche Aufschlüsselung des Strommixes, also wie viel Energie wann wirklich aus welcher Quelle kam (Wind, Solar, Biogas, Wasser, Import, Export...)

### Aufgabe 6 (Analyse von Stromtarif-Angeboten für Endkunden)
<a id = "aufgabe6"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)

a) Führen Sie die gegebenen Preisvergleichdaten in einem DataFrame namens df_cust zusammen. Exportieren Sie diesen als CSV-Datei namens prices_customers.csv und laden Sie diese mit Ihrer Abgabe auf Moodle hoch. Verwerfen Sie bitte zur Minimierung der Dateigröße alle Spalten, die im weiteren Verlauf nicht mehr verwendet werden. Kommentieren Sie nun den Code zur Datensatzgenerierung aus und lesen Sie die CSV-Datei in den DataFrame df_cust erneut aus dieser Datei ein.


In [ ]:
'''
path = './Daten/Endkundenpreise/'

def extract_date_from_filepath(file):
    file = file.replace(path, "")
    # Concatenate the date out of fixed year 2024 and month/day from folder name
    return "2024-" + file[0:5]

def extract_data_from_json(file):
    assert file.endswith(".json"), "Keine json-Datei uebergeben"
    try:
        # Read and load the json file
        df_temp = pd.read_json(file)
        # Transpose the table: convert the rows to columns
        df_temp = df_temp.T
        # Add date from filename as column
        df_temp['Datum'] = extract_date_from_filepath(file)
        
        return df_temp
    except:
        print("Datei konnte nicht gelesen werden.")

def drop_unnecessary_columns(df):
    columns_to_drop = ['Postleitzahl', 'Jahresverbrauch', 'Abschlagszahlung', 'Verlängerung', 'Kündigungsfrist', 'Grundpreis', 'Arbeitspreis', 'Preisgarantie', 'Grundpreisrabatt:', 'Neukundenbonus', 'Sofortbonus', 'Arbeitspreisrabatt', 'Zusätzlicher Aktionsbonus', 'Blitzbonus', 'Abschlagsrabatt', 'Grundpreisrabatt', 'Winterprämie']
    return df.drop(columns = columns_to_drop)

df_list = []
# https://www.tutorialspoint.com/python/os_listdir.htm
for folder in os.listdir(path):
    combined_path = os.path.join(path, folder, "*json")
    json_files = glob.glob(combined_path)

    for i in range(len(json_files)):
        df_temp = extract_data_from_json(json_files[i])
        df_list.append(df_temp)
        
df_cust = pd.concat(df_list)
df_cust = drop_unnecessary_columns(df_cust)
# https://www.datacamp.com/tutorial/save-as-csv-pandas-dataframe
df_cust.to_csv('prices_customers.csv', index = False, encoding='utf-8')
'''

b) Bereiten Sie die Daten auf die weitere Analyse vor, indem Sie geeignete Datentransformations- und -bereinigungsschritte durchführen.

In [ ]:
df_cust = pd.read_csv('prices_customers.csv')
print("Zeilen vor Bereinigung:", df_cust.shape[0])

df_cust['Datum'] = pd.to_datetime(df_cust['Datum'])

def combine_price_columns(row):
    if pd.isna(row['Preis im 1. Jahr*']):
        return row['Preis im 1. Jahr']
    return row['Preis im 1. Jahr*']

def replace_string_from_row(row, column, string, string_to_replace_with):
    if pd.isna(row[column]):
        return row[column]
    if string in row[column]:
        return row[column].replace(string, string_to_replace_with)
    return row[column]

# Combine the two price columns into one
df_cust['Preis'] = df_cust.apply(combine_price_columns, axis = 1)
df_cust = df_cust.drop(columns = ['Preis im 1. Jahr*', 'Preis im 1. Jahr'])
# Drop rows with NaN as Preis, because those rows aren't viable for a comparison later on
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
df_cust = df_cust.dropna(subset=['Preis'])

df_cust['Vertragslaufzeit'] = df_cust.apply(lambda row: replace_string_from_row(row, 'Vertragslaufzeit', " Monate", ""), axis=1)
df_cust['Vertragslaufzeit'] = df_cust.apply(lambda row: replace_string_from_row(row, 'Vertragslaufzeit', " Monat", ""), axis=1)
# Fill NaN values, so that dtype can be converted to int
df_cust['Vertragslaufzeit'] = df_cust['Vertragslaufzeit'].fillna(0)
df_cust['Vertragslaufzeit'] = df_cust['Vertragslaufzeit'].astype(int)

df_cust['Preis'] = df_cust.apply(lambda row: replace_string_from_row(row, 'Preis', " €/Monat", ""), axis=1)
# Change decimal comma, to decimal point for float conversion
df_cust['Preis'] = df_cust.apply(lambda row: replace_string_from_row(row, 'Preis', ",", "."), axis=1)
df_cust['Preis'] = df_cust['Preis'].astype(float)
df_cust = df_cust.rename(columns = {'Preis' : 'Preis im 1. Jahr/ pro Monat in €', 'Vertragslaufzeit' : 'Vertragslaufzeit in Monaten'})

# Entfernung des Zeilumbruchs zur besseren Ausgabe des Tarifnamens
df_cust['Tarif'] = df_cust.apply(lambda row: replace_string_from_row(row, 'Tarif', "\n", " "), axis = 1)
df_cust['Tarif'] = df_cust.apply(lambda row: replace_string_from_row(row, 'Tarif', "  ", " "), axis = 1)

print("Zeilen nach Bereinigung:", df_cust.shape[0])



In [ ]:
# Duplikate selber Tarif, selber  Anbieter mit dem selben Preis in der selben Stadt
df_not_unique = df_cust[df_cust.duplicated(subset=['Stadt','Datum','Tarif','Anbieter', 'Preis im 1. Jahr/ pro Monat in €'], keep = False)]

# 41 Duplikate?
df_not_unique = df_not_unique.sort_values(by=['Stadt', 'Datum'])

print("Vor der Bereinigung: ", df_cust.shape[0])

# gefundene Duplikate nur erstes Vorkommen behalten
df_cust = df_cust.drop_duplicates(subset=['Stadt','Datum','Tarif','Anbieter', 'Preis im 1. Jahr/ pro Monat in €'], keep = 'first')

print("Nach der Bereinigung: ", df_cust.shape[0])


c) Wie viele verschiedene Tarife wurden insgesamt angeboten? Zu wie vielen Tagen sind pro Stadt Daten vorhanden? Wie viele verschiedene Anbieter haben insgesamt Tarife angeboten?

In [ ]:
unique_tariffe = df_cust['Tarif'].unique()
print("Anzahl verschiedender Tarife:" , len(unique_tariffe))

# Occuring Städte in the DataSet
number_of_total_cities = len(df_cust['Stadt'].unique())

# Group with any aggregation to get all unique Datum/Stadt combination entries
grouped_date_city = df_cust.groupby(['Datum', 'Stadt']).agg(Count=('Stadt', 'count')).reset_index()
# Group and count how many different Stadt rows there are for each given date
grouped_date = grouped_date_city.groupby( ['Datum'])['Stadt'].count()
# Filter out the dates that don't have an entry for every Stadt of the DataSet
filtered_dates = grouped_date[grouped_date == number_of_total_cities]
print("Daten für alle Städte sind an", len(filtered_dates), "Tagen vorhanden")

unique_anbieter = df_cust['Anbieter'].unique()
print("Anzahl der Anbieter die insgesamt Tarife angeboten haben:", len(unique_anbieter))

d) Ermitteln Sie, welche unterschiedlichen Tarife in Amberg angeboten wurden und visualisieren Sie exemplarisch für die Stadt Amberg den Füllgrad der Daten. Erstellen Sie dazu eine HeatMap, aus der hervorgeht, an welchen Tagen es zu welchen der ermittelten Tarife Angebotsdaten gab.

In [ ]:
df_amberg = df_cust[df_cust['Stadt'] == 'Amberg']
df_amberg = df_amberg[['Datum', 'Tarif', 'Preis im 1. Jahr/ pro Monat in €']]
unique_tarife_amberg = df_amberg['Tarif'].unique()

print("Anzahl verschiedender Tarife:" , len(unique_tarife_amberg))

df_2024_daily = df_daily[df_daily['Datum'].dt.year == 2024]

df_2024 = df_2024_daily[['Datum']].copy()

# Setze den Index zurück und ändere den DataFrame inplace
df_2024.reset_index(drop=True, inplace=True)

# Füge eine neue Spalte 'Merge Key' mit dem Wert 1 für jede Zeile hinzu
df_2024['Merge Key'] = 1

df_tarife = pd.DataFrame({'Tarif' : unique_tarife_amberg})
df_tarife['Merge Key'] = 1
df_tarife


df_merged_2024 = pd.merge(df_2024, df_tarife, how = 'left', on = 'Merge Key').drop('Merge Key', axis=1)

print(df_merged_2024.shape)

df_amberg_final = pd.merge(df_merged_2024, df_amberg, how = 'left', on = ['Datum', 'Tarif'])
df_amberg_final = df_amberg_final.fillna(-1)
df_amberg_final['Indicator'] = df_amberg_final['Preis im 1. Jahr/ pro Monat in €'].apply(lambda x: 1 if x != -1 else 0)

df_temp = df_amberg_final.copy()

df_häufigkeit = df_amberg_final.groupby('Tarif')['Indicator'].sum()
df_häufigkeit.name ='Häufigkeit'
df_häufigkeit.reset_index()

df_amberg_final = pd.merge(df_amberg_final, df_häufigkeit, how = 'left', on = 'Tarif')
df_amberg_final = df_amberg_final.sort_values(by ='Häufigkeit', ascending = False)
df_amberg_final

In [ ]:
# Neue Spalte für Hovertext hinzufügen
df_amberg_final['HoverText'] = df_amberg_final['Indicator'].map({1: 'Vorhanden', 0: 'Nicht Vorhanden'})

trace = go.Heatmap(
    x = df_amberg_final['Datum'],
    y = df_amberg_final['Tarif'],
    z = df_amberg_final['Indicator'],
    showscale = False,
    hovertemplate = 
    'Datum: %{x} <br>'
    'Verfügbarkeit: %{customdata}',
    
    customdata = df_amberg_final['HoverText'],
    name='',
    colorscale='magma'
)

data = [trace]

layout = go.Layout(
    title=go.layout.Title(
        text='Tarife in Amberg',
    ),
    width = 1500,
    height = 800,
    
     xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Datum',
        )  
    ),
    
    yaxis= go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text= 'Tarife',
        )
    )
    
    
)

fig = go.Figure(data=data, layout=layout)
fig.show()

e) Visualisieren Sie die durchschnittliche Preisentwicklung im Verlauf des Jahres 2024 über alle Tarife und Orte hinweg. Berücksichtigen Sie dabei nur Tarife, bei denen die Vertragslaufzeit mindestens 12 Monate beträgt. Verwenden Sie dazu den Preis im 1. Jahr, der den monatlichen Preis unter Berücksichtigung des Grundpreises, des Arbeitspreises und von Bonuszahlungen o.ä. enthält. Untersuchen Sie anschließend den Zusammenhang zum Börsenstrompreis, indem Sie geeignete Kenngrößen berechnen und weitere Diagramme erstellen.

In [ ]:
filtered_12_months = df_cust[df_cust['Vertragslaufzeit in Monaten'] >= 12]
df_mean_prices = filtered_12_months.groupby(['Datum']).agg(MeanPrice =('Preis im 1. Jahr/ pro Monat in €', 'mean')).reset_index()

mean_price_total = df_mean_prices['MeanPrice'].mean()

df_mean_prices['Difference to Total Mean'] = (df_mean_prices['MeanPrice'] - mean_price_total)
# Constant value column for plotting the base line
df_mean_prices['Total Mean'] = 0

plt.figure(figsize=(20, 7))
# Mean/Base line
plt.plot(df_mean_prices['Datum'], df_mean_prices['Total Mean'], label = 'Durchschnittlicher Endkundenpreis 2024', color = 'gray', linestyle='--')
# Label for Mean/Base line
mean_line_label = str(round(mean_price_total,2)) + " €"
plt.text(df_mean_prices['Datum'].min(), 0.35, mean_line_label, color='gray', fontsize=12)

# Price Deviation line
plt.plot(df_mean_prices['Datum'], df_mean_prices['Difference to Total Mean'], label = 'Abweichung in €', color = 'blue')

#https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.ylim.html
# Center mean price, to balance out the appearance of change
plt.ylim(bottom = -9, top = 9)

plt.xlabel('2024', fontsize = 12) 
plt.ylabel('Abweichung', fontsize = 12)
plt.title('Durchschnittliche Endkundenpreisentwicklung im Vergleich zum durchschnittlichen Endkundenpreis 2024', fontsize=14)
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.MonthLocator())
plt.grid(True, linestyle = '--', alpha = 0.5)

plt.legend()
plt.show()

df_mean_prices = df_mean_prices.drop(columns = ['Difference to Total Mean', 'Total Mean'])


Vergleich Endkundenpreis zu Börsenstrompreis

In [ ]:
df_kunden_boerse = pd.merge(df_mean_prices, df_daily, on = 'Datum', how = 'left', validate='one_to_one')
df_kunden_boerse = df_kunden_boerse[['Datum','MeanPrice', 'Day Ahead Auktion Preis (EUR/MWh)']]
df_kunden_boerse = df_kunden_boerse.rename(columns = {'MeanPrice': 'Endkundenpreis pro 4000kWh (EUR/Monat)'})

# 1 MWh = 1.000 kWh
df_kunden_boerse['Börsenpreis pro kWh (EUR)'] = (df_kunden_boerse['Day Ahead Auktion Preis (EUR/MWh)'] / 1000)
df_kunden_boerse['Endkundenpreis pro kWh (EUR)'] = (df_kunden_boerse['Endkundenpreis pro 4000kWh (EUR/Monat)'] * 12) / 4000

plt.figure(figsize=(10, 10))
plt.scatter(df_kunden_boerse['Endkundenpreis pro kWh (EUR)'], df_kunden_boerse['Börsenpreis pro kWh (EUR)'], alpha=0.75)
plt.xlabel('Endkundenpreis pro kWh (EUR)', fontsize = 12) 
plt.ylabel('Börsenpreis pro kWh (EUR)', fontsize = 12)
plt.title('Vergleich Börsenstrompreis zu Endkundenstrompreis')
plt.show()

Es lässt sich aus dem Scatter-Plot ablesen, dass die teils extremen Schwankungen des Börsenpreises sich nicht im Endkundenpreis wiederfinden, also es keine Korrelation zwischen den beiden Preisen zu geben scheint. 

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(df_kunden_boerse['Datum'], df_kunden_boerse['Endkundenpreis pro kWh (EUR)'], label = 'Durchschnittlicher Preis pro kWh im 1. Jahr (Endkunde)', color = 'blue')
plt.plot(df_kunden_boerse['Datum'], df_kunden_boerse['Börsenpreis pro kWh (EUR)'] , label = 'Durchschnittlicher Preis pro kWh (Börse)', color = 'gray')
plt.ylabel('Preis pro kWh in €', fontsize = 12) 
plt.xlabel('2024', fontsize = 12)
plt.title('Vergleich der durchschnittlichen Strompreise pro kWh Börse/Endkunde') 
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.MonthLocator())
plt.grid(True, linestyle = '--', alpha = 0.5)
plt.legend()
plt.show()

Auch in diesem Plot lässt sich noch mal erkennen dass der Endkundenpreis von den Schwankungen quasi unbeeinträchtigt ist. Jedoch sieht man auch dass der Börsenpreis bis auf ein paar Ausnahmen weit unter dem Endkundenpreis läuft.

In [ ]:
df_kunden_boerse['Marge Anbieter'] = df_kunden_boerse['Endkundenpreis pro kWh (EUR)'] - df_kunden_boerse['Börsenpreis pro kWh (EUR)'] 

plt.figure(figsize=(15, 5))
plt.plot(df_kunden_boerse['Datum'], df_kunden_boerse['Marge Anbieter'], color = 'blue')
plt.ylabel('Marge in €', fontsize = 12)
plt.xlabel('2024', fontsize = 12)
plt.title('Durchschnittliche Marge der Anbieter pro kWh')
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.MonthLocator())
plt.grid(True, linestyle = '--', alpha = 0.5)

plt.ylim(-0.3, 0.3)
plt.show()

print("Minimale Marge: " + str(round(df_kunden_boerse['Marge Anbieter'].min(),2)) + "€")
print("Maximale Marge: " + str(round(df_kunden_boerse['Marge Anbieter'].max(),2)) + "€")
print("Durchschnittliche Marge: " + str(round(df_kunden_boerse['Marge Anbieter'].mean(),2)) + "€")

Die Marge des Anbieters ist – mit wenigen Ausnahmen – durchweg positiv. Selbst bei extremen Ausschlägen auf dem Strommarkt gibt der Anbieter die hohen Preise nicht vollständig an die Kunden weiter, sondern nimmt die Verluste in Kauf. Dennoch ist Mitleid mit den Stromanbietern nicht angebracht: Die durchschnittliche Marge von 0,19 € pro Einheit kompensiert diese Verluste über die 12-monatige Laufzeit hinweg. Dies gilt jedoch unter der Annahme, dass solche extremen Preissprünge tatsächlich nur vorübergehende Ausnahmen bleiben und sich der Börsenstrompreis nicht dauerhaft auf diesem hohen Niveau stabilisiert.

f) Erstellen Sie ein interaktives Liniendiagramm in Plotly, um die zeitliche Entwicklung der Angebotspreise pro Stadt zu visualisieren. Das Diagramm soll eine Dropdown-Liste enthalten, über die man die Stadt auswählen kann. Das Diagramm soll die Preisentwicklung für diejenigen zehn Tarife zeigen, die in der jeweiligen Stadt im Verlauf des Jahres am häufigsten angeboten wurden (die sich also am häufigsten unter den günstigsten 20 Tarifen beim Preisvergleich befanden). Durch Klick auf den Namen des Tarifs in der Legende soll es möglich sein, dessen Kurve im Diagramm ein- und auszublenden.

Zur Visualisierung der 10 häufisten Tarife pro Stadt gruppieren wir nach der Spalte Stadt und der Spalte Tarif und zählen diese. Das Ergebnis ist dann eine Series mit einem Multindex. 
Mit der Methode n.largest(10) werden die 10 größten Werte zurückgegeben. 
Um den ursprünglichen DataFrame und unserer Series zu mergen, setzen wir den Index zurück und mergen über die Spalten Stadt und Tarif. Anschließend werden die Zeilen mit NaN gedroppt. Zur übersichtlicheren weiteren Arbeit mit dem Dataframe entfernen wir die Spalten Anbieter und den vorherig berechneten Tarif_Count. 

In [ ]:
df_citys = df_cust.groupby(['Stadt', 'Tarif'])['Tarif'].count()
df_citys.name = 'Tarif_Count'

top_10_per_city = df_citys.groupby('Stadt', group_keys=False).nlargest(10)
df_top_10 = top_10_per_city.reset_index()


df_top_10_final = pd.merge(df_cust, df_top_10, how = 'left', on = ['Stadt', 'Tarif']).dropna().reset_index(drop=True)
df_top_10_final = df_top_10_final.drop(columns = ['Tarif_Count', 'Anbieter'])

In [ ]:
fig = go.Figure()
df_top_10_final = df_top_10_final.sort_values(by='Stadt')
cities = df_top_10_final['Stadt'].unique()

trace_list = []

for city in cities:
    city_data = df_top_10_final[df_top_10_final['Stadt'] == city]
    
    #trace (line) for each tarif and city
    for tarif in city_data['Tarif'].unique():
        tarif_city_data = city_data[city_data['Tarif'] == tarif]
        # Sort values by date, so that trace connects the rows in the right order
        tarif_city_data = tarif_city_data.sort_values(by='Datum')
  
        #https://plotly.com/python/reference/scatter/
        
        trace = go.Scatter(
            x=tarif_city_data['Datum'],
            y=tarif_city_data['Preis im 1. Jahr/ pro Monat in €'],
            mode='lines',
            name=f'{tarif}',
            meta = city,
            # initally all traces are hidden
            visible=False,
            
            hovertemplate =
            'Datum: %{x} <br>' 
            'Preis: %{y}€'
            # https://plotly.com/python/hover-text-and-formatting/
             '<extra></extra>',
        )
        
        trace_list.append(trace)
        fig.add_trace(trace)



def visible(city):
    visibility = []
    for trace in fig.data:
        if city in trace.meta:
            visibility.append(True)
        else:
            visibility.append(False) 
    return visibility  

# Showing Amberg first

visibility_flags = visible('Amberg')

for index, trace in enumerate(fig.data):
    trace.visible = visibility_flags[index]

    

# Add the dropdown menu for selecting the city

# https://stackoverflow.com/questions/66414456/update-visibility-of-traces-with-fig-update-layout-plotly

fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                {
                    'label': city,
                    'method': 'update',
                    'args': [
                        {'visible': visible(city)}
                    ]
                } for city in cities
            ],
            direction='down',
            showactive=True,
            x=-0.08,
            xanchor='right',
            y=1,  
            yanchor='top'
        )
        
    ]
)

y_min = df_top_10_final['Preis im 1. Jahr/ pro Monat in €'].min()
y_max = df_top_10_final['Preis im 1. Jahr/ pro Monat in €'].max()
fig.update_layout(
    autosize=False, 
    width=1550,      
    height=500,
    title = 'Preise der 10 häufigsten Tarife pro Stadt für 2024',
    xaxis_title="Datum",
    yaxis_title="Endkundenpreis in €",
    yaxis = dict(
        range = [y_min, y_max]
    )
)


fig.show()

g) Untersuchen Sie mit Hilfe des Diagramms die Preisentwicklung der verschiedenen Anbieter in Amberg. Welche Empfehlungen leiten Sie für den Abschluss eines neuen Vertrags ab?

Grundsätzlich zeigt sich beim Vergleich der 10 häufigsten Tarife in Amberg für das Jahr 2024, dass sich ein Preisvergleich immer lohnt, um den besten Preis zu finden.
Sichtbar wird, dass sich vor allem ab der Jahresmitte die Endkundenpreise der verschieden Tarife in 2 Richtungen entwickeln, wodurch sich ein paar teurere und ein paar günstigere Tarife für den Rest des Jahres ergeben. Unsere Schlussfolgerung aus dem Jahr 2024 ist dementsprechend, dass sich vor allem zur Jahreshälfte ein Preisvergleich lohnen kann. 

Die günstigsten Tarife 2024 waren hierbei der Tarif rabot.home flex, der Tarif Fairpower X und der Tarif Dynmaischer Tarif. Diese könnten potentiell auch im nächsten Jahr günstiger sein. 

h) Untersuchen Sie die durchschnittlichen Preisniveaus pro Stadt und visualisieren Sie diese auf einer Karte in Folium. Lassen sich bestimmte Trends und Einflussfaktoren erkennen?

Wir haben die Koordinaten der Städte rausgesucht und im CSV-File 'coordinates.csv' abgespeichert. Dazu haben wir auch noch den Regierungsbezirk und die Einwohnerzahl (Stand 2022/2024) der Städte eingetragen, um potenzielle Einflussfaktoren auf den Strompreis zu überprüfen.

In [ ]:
df_cities_coordinates = pd.read_csv('coordinates.csv')

df_city_price_mean = df_cust.groupby(['Stadt']).agg(Durchschnittspreis=('Preis im 1. Jahr/ pro Monat in €', 'mean')).reset_index()
df_city_price_mean['Durchschnittspreis'] = df_city_price_mean['Durchschnittspreis'].round(2)

df_city_price_map = pd.merge(df_city_price_mean, df_cities_coordinates, on = 'Stadt', how = 'left', validate='one_to_one')

# Start map at mean Location of all the data rows
prices_map = folium.Map(location=[df_cities_coordinates['Latitude'].mean(), df_cities_coordinates['Longitude'].mean()])

# Circle Marker red if the city is more expensive than the average, red if less
total_mean_price = df_cust['Preis im 1. Jahr/ pro Monat in €'].mean().round(2)
print('Bayrischer Durchschnittspreis: ' + str(total_mean_price) + '€')
def decide_color_for_marker(price):
    if price >= total_mean_price:
        return 'red'
    return 'green'

print("Grün: Billiger als der Durchschnitt, Rot: Teurer als der Durchschnitt")
# https://python-visualization.github.io/folium/latest/user_guide/vector_layers/circle_and_circle_marker.html
for index, row in df_city_price_map.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius= 10,
        color=decide_color_for_marker(row['Durchschnittspreis']),
        fill=True,
        fill_color=decide_color_for_marker(row['Durchschnittspreis']),
        fill_opacity=1,
        popup=f"{row['Stadt']}: {row['Durchschnittspreis']} Durchschnittspreis",
        tooltip=f"{row['Stadt']}, {row['Durchschnittspreis']} €"
    ).add_to(prices_map)

prices_map

Man kann in der Karte erkennen, dass in der Region der Oberpfalz viele Kreise rot sind und damit teurer als der bayrische Durchschnitt.
Heißt das die Oberpfalz hat einfach teuren Strom?

In [ ]:
# Filtering out outliers (Großstädte) --> makes linear correlation easier/possible
# especially München and Nürnberg are just too big to correlate this way
# dropping München, Nürnberg, Ausburg, Ingolstadt, Würzburg, Erlangen and Regensburg
df_city_price_map_without_outliers = df_city_price_map.copy()
df_city_price_map_without_outliers = df_city_price_map_without_outliers[(df_city_price_map['Einwohner'] <= 100_000)]

plt.scatter(df_city_price_map_without_outliers['Einwohner'],df_city_price_map_without_outliers['Durchschnittspreis'])
plt.ylabel('Durchschnittler Endverbraucherpreis €')
plt.xlabel('Einwohner')
plt.title('Zusammenhang Strompreis und Einwohner (ohne Großstädte)')
plt.show()

# Show correlation between Einwohner and Preis
df_city_price_map_corr = df_city_price_map_without_outliers[['Einwohner', 'Durchschnittspreis']]
print(df_city_price_map_corr.corr())

Wir haben noch die Einwohner der Städte betrachtet, und überlegt ob Einwohnerzahl und Strompreis für den Kunden zusammenhängen. 
Und tatsächlich, wenn man die Großstädte für die Korrelation herausfiltert, erkennt man einen Trend, das besonders kleine Orte und Städte teureren Strom haben als größere Städte. Also hat die Oberpfalz vielleicht in diesem Datensatz einfach kleinere Städte, die den Durchschnittspreis nach oben treiben?

In [ ]:
df_regierungsbezirk = df_city_price_map.groupby('Regierungsbezirk').agg(
    Durchschnittspreis=('Durchschnittspreis', 'mean'),
    DurchschnittlicheEinwohner=('Einwohner', 'mean')
).reset_index()

df_regierungsbezirk = df_regierungsbezirk.sort_values(by='Durchschnittspreis', ascending=False)

plt.figure(figsize=(15, 5))
plt.bar(df_regierungsbezirk['Regierungsbezirk'], df_regierungsbezirk['Durchschnittspreis'])
plt.ylabel('Durchschnittlicher Endkundenpreis €')
plt.title('Preisvergleich der bayrischen Regierungsbezirke (mit Großstädten)')
plt.show()

df_regierungsbezirk

Die Oberpfalz weist in diesem Datensatz eine Vielzahl kleiner Orte und Städte auf und hat mit Abstand die geringste durchschnittliche Einwohnerzahl. Nach unseren Beobachtungen trägt dies dazu bei, die Strompreise nach oben zu treiben. Dies zeigt sich auch auf der Karte, wo nur die größeren Städte der Oberpfalz, wie Regensburg, Amberg und Weiden, grün markiert sind, da sie günstigere Strompreise aufweisen.

In Schwaben und Oberbayern hingegen sind die Strompreise ebenfalls hoch. Besonders auffällig ist dies in München/Kempten und deren Umgebung, wo trotz der Größe der Stadt die Stromkosten vergleichsweise teuer bleiben, das heißt auch die Lage beeinflusst die Strompreise, dabei kann man in der Karte herauslesen, dass der Süden Bayerns tendenziell den teureren Strom hat (im Vergleich zum Norden)

i) Im Merkmal Anbieter befinden sich kurze Beschreibungen der Anbieter und der Tarife. Erstellen Sie mit Hilfe des Pakets WordCloud eine Wortwolke für die Anbieter-Beschreibungen und untersuchen Sie, welche Schlagworte besonders häufig auftreten.

Wir splitten die Anbieter-Beschreibungen auf, entfernen Satzzeichen und Füllwörter und bauen dann daraus die WordCloud.

In [ ]:
def remove_punctuation_from_anbieter(anbieter):
    punctuations = [
        ".", ",", "/", "&", "-", "_", ":", ";", "!", "?", "#", "%", "$", "@", "^", "+",
        "=", "(", ")", "[", "]", "{", "}", "<", ">", "|", "~", "'", "`"]
    for p in punctuations:
        anbieter = anbieter.replace(p, "")
    return anbieter
    
def remove_non_buzzwords_from_anbieter(anbieter):
    anbieter_words = anbieter.split() # Isolate each word of Anbieter

    words_to_remove = [
        'der', 'die', 'das', 'des', 'und', 'ein', 'eine', 'einen', 'mit', 'ist', 'den', 'dem', 'zu', 'von', 'vom', 
        'auf', 'im', 'an', 'für', 'am', 'als', 'es', 'aber', 'auch', 'aus', 'bei', 'dass', 'um'
        'du', 'er', 'sie', 'wir', 'ihr', 'ihnen', 'ihm', 'euch', 'mir', 'mich', 
        'mein', 'meine', 'dein', 'deine', 'sein', 'seine', 'ihr', 'ihre', 
        'noch', 'schon', 'oder', 'so', 'wie', 'was', 'wer', 'wenn', 'beim'
        'warum', 'weil', 'dann', 'doch', 'nur', 'diese', 'dieser', 'dieses', 
        'jeder', 'jede', 'jedes', 'keiner', 'keine', 'kein', 'welche', 'welcher', 
        'man', 'damit', 'über', 'unter', 'haben', 'hat', 'sein', 'sind', 'war', 'waren', 
        'dabei', 'in', 'seit', 'durch', 'ihren', 'einer', 'sowie', 'gmbh', 'kg', 'ag']

    filtered_words = []
    
    for word in anbieter_words:
        # Only keep words that are buzzwords
        if word.lower() not in words_to_remove:
            filtered_words.append(word)

    # https://www.w3schools.com/python/ref_string_join.asp
    return ' '.join(filtered_words)

# Take every Anbieter Description once
unique_anbieter_strs = df_cust['Anbieter'].unique()

for i in range(len(unique_anbieter_strs)):
    unique_anbieter_strs[i] = remove_punctuation_from_anbieter(unique_anbieter_strs[i])
    unique_anbieter_strs[i] = remove_non_buzzwords_from_anbieter(unique_anbieter_strs[i])

# Create one String out of all different Anbieter 
combined_text = ' '.join(unique_anbieter_strs)

cloud = WordCloud(background_color='white').generate(combined_text)
plt.figure(figsize=(12,5))
plt.imshow(cloud)
plt.axis('off')
plt.show()


Die häufigsten Wörter, wie "Strom" und "Energie", sind wenig überraschend und unterstreichen den Fokus der Anbieter auf ihr Kerngeschäft. Auffällig ist jedoch, dass auch gezielt Schlagwörter wie "Kunden", "bieten", "Ökostrom" und "versorgt" verwendet werden. Diese Begriffe sollen potenziellen Kunden gefallen und sie dazu bewegen, sich für den Tarif des jeweiligen Anbieters statt für die Konkurrenz zu entscheiden.

### Quellenverzeichnis
<a id = "quellenverzeichnis"></a>

[Zurück zum Inhaltsverzeichnis](#inhaltsverzeichnis)